# Using external inputs to interrogate a model of single neuron

## Useful references

### Python + Numpy + Matplotlib + etc.

* Python Numpy Tutorial: http://cs231n.github.io/python-numpy-tutorial/
* Numpy for MATLAB users: https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html
* MATLAB synonymous commands in Python/NumPy: http://mathesaurus.sourceforge.net/
* Effectively using matplotlib: https://pbpython.com/effective-matplotlib.html


### NEURON (with Python)
* NEURON documentation: https://www.neuron.yale.edu/neuron/static/py_doc/index.html
* NEURON + Python tutorial: https://neuron.yale.edu/neuron/docs/scripting-neuron-basics


Here we demonstrate how to use external inputs to a model neuron to characterize and study its properties.

For this, we need objects to deliver localized perturbation to a neuron, which we call [point processes](https://neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/mech.html). Today, we will particularly focus on the [`IClamp`](https://neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/mech.html#IClamp) that can perform current injection into a neuron --- a simulation of the current clamp experiment.

We first start with importing necessary modules, including `neuron`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h, gui

Then, we load a model of a cortical pyramidal cell model

In [ ]:
import libcell

cell = libcell.L23()

We can inspect the model by `psection`

In [ ]:
h.psection()

In [ ]:
cell.dends[0].psection()

In [ ]:
cell.dends

Here we load a session file for the virtual current clamp experiment:

In [ ]:
h.xopen("iclamp.ses")

Loading this session embeds a current clamp electrode `h.IClamp[0]`, whose parameters are:

In [ ]:
print('Activation onset = {} ms\nDuration = {} ms\nAmplitude = {} nA'.format(
        h.IClamp[0].delay, h.IClamp[0].dur, h.IClamp[0].amp))

You can control the location and parameters via the "PointProcessManager" window. Now let's run simulation while recording the membrane potential at soma.

In [ ]:
def run(tstop=300):
    """runs a simulation and returns the membrane potential recording at the soma."""
    dt = 0.1 # Again we use 10kHz sampling rate, e.g. 0.1 ms interval
    vrec = h.Vector() # The recording will be save in this vector
    vrec.record(cell.soma(0.5)._ref_v, dt)
    
    h.tstop = tstop  # Set how long the simulation will run.
    h.init()
    h.run()
    
    return vrec.c() # Should return a copy of the vector


Let's collect multiple recordings in a list and also make a note about where the electrode was. First, we prepare two empty lists, one for voltage recordings and the other for electrode-to-soma distance:

In [ ]:
# Reset lists
vrecs = []
distances = []

# Set the origin to cell.soma(0) 
# See https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/topology/geometry.html#distance
h.distance(sec=cell.soma) 

In [ ]:
result = run()
vrecs.append(result)

# Measure a distance to the origin from the electrode
distances.append(
    h.distance(h.IClamp[0].get_segment())
) 
print("Number of simulations ran =", len(vrecs), 
     ", Distance = ", h.distance(h.IClamp[0].get_segment()), "um")

In [ ]:
fig,ax = plt.subplots()

t = np.arange(3000)*0.1 # Remember that we use 10kHz for a sampling rate
for v in vrecs:
    ax.plot(t, v)
ax.legend(distances)
ax.set(xlabel='time (ms)', ylabel='Voltage (mV)')

## * How to locate and move point processes

### 1. How to find out the electrode location

`PointProcess.get_segment()` returns the segment that a point process is located.



In [ ]:
sec = h.IClamp[0].get_segment()
sec

Now we can measure the distance between any section(0) to this electrode using `h.distance`. For example,

In [ ]:
h.distance(sec=cell.soma) # Set an origin to cell.soma(0)
h.distance(h.IClamp[0].get_segment()) # Measure a distance to the origin from the electrode

### 2. How to find and get a particular section/segment

Here's how find all sections and segments in our `cell`:
* `h.allsec()` yields an iterator to go through all the sections in a model,
* Given a section, say `sec`, `sec.allseg()` yields an iterator for all segments in `sec`,
* If we know a particular location in terms of the relative coordinate `x`, we can use `sec(x)`,

Here is an example of going through all segments in a model with showing their distance to the soma:

In [ ]:
print('segment , distance')
for sec in h.allsec():
    for seg in sec.allseg():
        segname = '{}({})'.format(sec.name(), seg.x)
        h.distance(sec=cell.soma)
        print(segname, ',', h.distance(seg))

`PointProcess.loc(segment)` moves the point process to a given segment.

In [ ]:
h.IClamp[0].loc(h.dend1_111(0.07142857142857142))

### 3. Example: How to find a segment in `cell` by a name and move our electrode to it.

In [ ]:
sec_to_find = [sec for sec in cell.dends if sec.name()=='dend2_12222']
sec_to_find = sec_to_find[0]
sec_to_find

In [ ]:
segs = [seg for seg in sec_to_find.allseg()]
segs

Then, we move our `IClamp` to the second segment,

In [ ]:
h.IClamp[0].loc(segs[7])

Or, we can move it to a middle segment of the section,

In [ ]:
h.IClamp[0].loc(sec_to_find(0.5))

### 4. How to access variables in sections

You can access a variable defined in a section by (section).(variable name). Also, you can change its value by assignment. It is also possible to access/change a variable only for a segment, if a relative coordinate `x` for the segment is known, by (section)(x).(variable name).

In the following example, we go through all the sections in the model and print the name, `g_pas` (conductance parameter of the passive mechanism), and `nseg`. 

In [ ]:
for sec in h.allsec():
    print(sec.name(), sec.g_pas, sec.nseg)

In [ ]:
for sec in h.allsec():
    for seg in sec.allseg():
        h.distance(sec=cell.soma)
        print(sec.name(), h.distance(seg), seg.v)

In [ ]:
h.psection()

Now let's change `g_pas`, the passive conductance, and check the result.

In [ ]:
cell.soma.g_pas = 0.0001
h.psection()